In [9]:
# Initialize and load the model
model = HybridModel(num_classes=2)
model = nn.DataParallel(model)
model.load_state_dict(torch.load('runs1/best.pt'))

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


<All keys matched successfully>

In [11]:
def get_total_parameters(model):
    return sum(p.numel() for p in model.parameters())

def get_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Calculate parameters
total_params = get_total_parameters(model)
trainable_params = get_trainable_parameters(model)
print(f"Total number of parameters in original model: {total_params}")
print(f"Number of trainable parameters in original model: {trainable_params}")

Total number of parameters in original model: 423410282
Number of trainable parameters in original model: 423410282


In [12]:
final_model_size=os.path.getsize('runs1/best.pt')
print(f"Size of 'best.pt': {final_model_size / (1024 ** 2):.2f} MB")

Size of 'best.pt': 1616.32 MB


In [8]:
import torch
import torch.nn as nn
import torch.quantization
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune
from tqdm import tqdm

def apply_pruning(model):
    for name, module in model.named_modules():
        if isinstance(module, (nn.Linear, nn.Conv2d)):
            prune.l1_unstructured(module, name="weight", amount=0.2)
            prune.remove(module, 'weight')  
    return model

def apply_quantization(model, train_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    if isinstance(model, torch.nn.DataParallel):
        model = model.module

    model.eval()  # Set the model to evaluation mode

    qconfig = torch.quantization.QConfig(
        activation=torch.quantization.default_observer.with_args(dtype=torch.qint8),
        weight=torch.quantization.default_weight_observer.with_args(dtype=torch.qint8)
    )
    
    model.qconfig = qconfig

    # Prepare the model for quantization
    torch.quantization.prepare(model, inplace=True)

    # Calibrate the model with training data
    with torch.no_grad():
        for inputs, _ in tqdm(train_loader, desc="Gathering statistics"):
            inputs = inputs.to(device)
            model(inputs)  # Forward pass 

    # Convert the model to a quantized version
    torch.quantization.convert(model, inplace=True)

    return model



# Apply pruning
model_pruned = apply_pruning(model)

model_pruned = nn.DataParallel(model_pruned)

# Apply post-training quantization
model_quantized = apply_quantization(model_pruned, train_loader)

# Save the quantized model
torch.save(model_quantized.state_dict(), 'runs1/quantized_model.pt')


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Gathering statistics: 100%|██████████| 728/728 [05:05<00:00,  2.38it/s]


RuntimeError: Quantized cudnn conv2d is currently limited to groups = 1; received groups =64

In [ ]:
import torch
import time
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast

def fine_tune(model, num_epochs, criterion, optimizer, train_loader, valid_loader, scaler):
    best_loss = float('inf')
    model.to(device)
    model.train()
    history = []
    early_stop = 0
    patience = 3

    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0.0
        predicted_labels = []
        true_labels = []
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)

        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                logits = model(inputs)
                loss = criterion(logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            progress_bar.set_postfix(loss=running_loss / len(progress_bar))

        validator_loss, metrics = validate_model(model, valid_loader, criterion)
        torch.cuda.empty_cache()

        precision, recall, f1 = metrics['precision'], metrics['recall'], metrics['f1']

        epoch_time = time.time() - start_time
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Time: {epoch_time:.2f} seconds")
        print(f'Epoch: {epoch + 1} Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, Loss: {validator_loss}')
        print("Confusion Matrix:")
        plot_matrix(metrics['confusion_matrix'])

        history.append({
            'Epoch': epoch + 1,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'Loss': validator_loss
        })

        if validator_loss < best_loss:
            best_loss = validator_loss
            torch.save(model.state_dict(), 'quantization/best_quantized.pt') 
            early_stop = 0
        else:
            early_stop += 1

        if early_stop >= patience:
            print("Early stopping triggered")
            break

    torch.save(model.state_dict(), 'quantization/final_quantized.pt')  

# Fine-tuning
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6, weight_decay=1e-5)
scaler = GradScaler()
fine_tune(model_quantized, 2, criterion, optimizer, train_loader, valid_loader, scaler) 